# B.A.S.I.C. - Bank Assistant for Service, Inquiries, and Care 🏦🤖

This notebook implements a prototype of a Retrieval-Augmented Generation (RAG) system designed for customer support at a fictional bank.

**Key Features:**
- Open-source LLM (≤6B parameters)
- Retrieval from structured and unstructured sources (JSON, Excel)
- Real-time document updates
- Friendly, domain-specific, and secure answers
- Guardrails against misuse or leakage

---

### ✅ Tasks Covered in This Notebook:
1. Data Ingestion and Preprocessing
2. Embedding Creation and Indexing
3. Query-Based Retrieval
4. Response Generation via LLM
5. Simple UI with live updates
6. Architecture Diagram


```
Syed Ahsan Ullah Tanweer - 368319
Navaira Rehman - 371063
```

# Step 1: Set up Environment

We install the necessary libraries for the project.

In [7]:
!pip install -q transformers datasets faiss-cpu sentence-transformers openpyxl gradio